In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 16.5 MB/s eta 0:00:00


In [2]:
#import sys
#!{sys.executable} -m pip install scikit-learn-intelex

In [3]:
#from sklearnex import patch_sklearn
#patch_sklearn(global_patch=True)

In [4]:
import sys
!{sys.executable} -m pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.1 MB/s eta 0:00:00


In [5]:
import sys
!{sys.executable} -m pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=86a5540d647049e26e6e9dec71c66c1c6b45c3834d890242130b4f1a15ca566c
  Stored in directory: /home/idies/.cache/pip/wheels/20/7c/09/4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.29
    Uninstalling GitPython-3.1.29:
      Successfully uninstalled GitPython-3.1.29


In [6]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [7]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='Random Forest',job_type='analysis',reinit=True)

wandb: Currently logged in as: sakhawat. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in /home/idies/workspace/Temporary/s.hossain18/scratch/jobs/20230718/20230718225212-245032/wandb/run-20230718_225329-4at9p8p5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Random Forest
wandb: ⭐️ View project at https://wandb.ai/sakhawat/exoplanetml
wandb: 🚀 View run at https://wandb.ai/sakhawat/exoplanetml/runs/4at9p8p5


In [8]:
import numpy as np

In [9]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [10]:
#from sklearn.model_selection import RepeatedKFold
#cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=41)

In [11]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [12]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier

In [13]:
rf_classifier = RandomForestClassifier(random_state=43,n_jobs = -1)

In [14]:
# Define the parameter grid
params = {
'max_depth': (150, 200),
'min_samples_split': (12, 20),
'min_samples_leaf': (5, 10),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy'],
'n_estimators': (400, 500)
}

In [15]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [16]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(rf_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [17]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=41),
              estimator=RandomForestClassifier(n_jobs=-1, random_state=43),
              n_iter=20, n_jobs=-1,
              search_spaces={'criterion': ['gini', 'entropy'],
                             'max_depth': (150, 200),
                             'max_features': ['sqrt', 'log2', None],
                             'min_samples_leaf': (5, 10),
                             'min_samples_split': (12, 20),
                             'n_estimators': (400, 500)})

In [18]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

Train score: 0.7963045714769854
Test score: 0.8013293943870015
Best hyperparameters: OrderedDict([('criterion', 'gini'), ('max_depth', 192), ('max_features', None), ('min_samples_leaf', 5), ('min_samples_split', 12), ('n_estimators', 400)])


In [19]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [20]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:  Test score ▁
wandb: Train score ▁
wandb: 
wandb: Run summary:
wandb:  Test score 0.80133
wandb: Train score 0.7963
wandb: 
wandb: 🚀 View run Random Forest at: https://wandb.ai/sakhawat/exoplanetml/runs/4at9p8p5
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230718_225329-4at9p8p5/logs
